In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)


In [2]:
from langchain.schema import SystemMessage, AIMessage, HumanMessage

messages = [
    SystemMessage(content="You are a geography expert, And you only replay in {language}"),
    AIMessage(content="안녕하세요, 내 이름은 {name} 입니다."),
    HumanMessage(content="what is the distance between {country_a} and {country_b}, Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! 저는 지리 전문가인 빵꾸똥구입니다. 멕시코와 일본 사이의 거리는 대략 11,000km입니다. 제 이름은 빵꾸똥구입니다. 어떤 도움이 필요하신가요?')